# Tutorial del Asistente de Chat

Esta notebook está diseñada para explicar el funcionamiento del archivo [`chat_assistant_2.py`](./chat_assistant_2.py), 
el cual implementa la lógica principal para interactuar con un agente de IA usando la API de OpenAI.

### Objetivos
- Entender la estructura general del código.
- Revisar las **clases principales** y su responsabilidad dentro del asistente.
- Analizar **funciones de utilidad** clave que apoyan el flujo de interacción.
- Ver **ejemplos prácticos** de cómo usar las clases y funciones en un entorno real.


> **Nota:** Este documento está pensado para aprendizaje y comprensión del código, 
no para ejecución en producción.


El script `chat_assistant_2.py` implementa una arquitectura modular para interactuar con un modelo de OpenAI, gestionar herramientas (funciones) 
y mostrar las respuestas de forma enriquecida en un entorno como **IPython** o **Jupyter Notebook**.

In [1]:
import json
import inspect
import markdown
from IPython.display import display, HTML

from openai import OpenAI

### Funciones de utilidad

#### `shorten(text, max_length=50)`
Acorta un texto si supera la longitud máxima definida.  
- Si la longitud del texto es menor o igual al límite, lo devuelve tal cual.  
- Si es mayor, recorta el texto y agrega `"..."` al final.  
Se utiliza principalmente para mostrar parámetros o argumentos de forma abreviada.

In [2]:
def shorten(text, max_length=50):
    if len(text) <= max_length:
        return text

    return text[:max_length - 3] + "..."

#### `generate_description(function)`
Genera un **esquema de descripción en formato JSON** para una función dada.  
Pasos:
1. Obtiene el nombre y docstring de la función.
2. Inspecciona su firma (`signature`) para extraer parámetros y tipos.
3. Mapea los tipos de Python a tipos de JSON Schema.
4. Marca todos los parámetros como requeridos salvo los que tengan valores por defecto.  

Esto es útil para registrar funciones como **herramientas** utilizables por el modelo de OpenAI.

In [3]:
def generate_description(function):
    """
    Generate a tool description schema for a given function using its docstring and signature.
    """

    # Get function name and docstring
    name = function.__name__
    doc = inspect.getdoc(function) or "No description provided."

    # Get function signature
    sig = inspect.signature(function)
    properties = {}
    required = []

    for param in sig.parameters.values():
        param_name = param.name
        param_type = param.annotation if param.annotation != inspect._empty else str

        # Map Python types to JSON schema types
        type_map = {
            str: "string",
            int: "integer",
            float: "number",
            bool: "boolean",
            dict: "object",
            list: "array"
        }
        json_type = type_map.get(param_type, "string")  # default to string

        properties[param_name] = {
            "type": json_type,
            "description": f"{param_name} parameter"
        }

        # Consider all parameters required unless they have a default
        if param.default == inspect._empty:
            required.append(param_name)

    return {
        "type": "function",
        "name": name,
        "description": doc,
        "parameters": {
            "type": "object",
            "properties": properties,
            "required": required,
            "additionalProperties": False
        }
    }

### Clase `Tools`

Gestiona un conjunto de funciones (herramientas) que el modelo puede invocar.

#### `__init__(self)`
Inicializa dos diccionarios:
- `tools`: guarda las descripciones de las funciones.
- `functions`: guarda las referencias a las funciones reales.

#### `add_tool(self, function, description=None)`
Registra una función como herramienta.  
- Si no se pasa `description`, la genera automáticamente con `generate_description`.
- Guarda la descripción y la referencia de la función.

#### `add_tools(self, instance)`
Registra **todos los métodos públicos** de una instancia como herramientas, usando `add_tool`.

#### `get_tools(self)`
Devuelve una lista con las descripciones de todas las herramientas registradas.

#### `function_call(self, tool_call_response)`
Ejecuta una función registrada a partir de la respuesta del modelo.  
1. Lee el nombre y argumentos de la función desde `tool_call_response`.
2. Llama a la función con esos argumentos.
3. Devuelve un diccionario con el resultado en formato JSON, incluyendo el `call_id`.

In [4]:
class Tools:

    def __init__(self):
        self.tools = {}
        self.functions = {}
    
    def add_tool(self, function, description=None):
        """
            tool_description = {
                "type": "function",               # This identifies it as a function/tool.
                "name": "<function_name>",       # Name of the function as it will be exposed.
                "description": "<function_description>",  # A human-readable description of the function's purpose.
                "parameters": {
                    "type": "object",            # Indicates the function accepts a JSON object as input.
                    "properties": {
                        "<param_name>": {
                            "type": "<type>",    # e.g., "string", "integer", etc.
                            "description": "<description>"  # Human-readable parameter description.
                        },
                        ...
                    },
                    "required": ["<param1>", "<param2>"],  # List of required parameters.
                    "additionalProperties": False          # Disallows additional unexpected parameters.
                }
            }
        """
        if description is None:
            description = generate_description(function)
        self.tools[function.__name__] = description
        self.functions[function.__name__] = function

    def add_tools(self, instance):
        for name, method in inspect.getmembers(instance, predicate=inspect.ismethod):
            if not name.startswith("_"):  # skip private and special methods
                self.add_tool(method)
    
    def get_tools(self):
        return list(self.tools.values())

    def function_call(self, tool_call_response):
        args = json.loads(tool_call_response.arguments)

        f_name = tool_call_response.name
        f = self.functions[f_name]
        
        call_id = tool_call_response.call_id
    
        results = f(**args)
        output_json = json.dumps(results)
        
        call_output = {
            "type": "function_call_output",
            "call_id": call_id,
            "output": output_json,
        }
    
        return call_output



### Clase `IPythonChatInterface`

Proporciona una interfaz para interactuar con el usuario y mostrar contenido en IPython/Jupyter.

#### `input(self)`
Solicita texto al usuario por consola.

#### `display(self, content)`
Imprime contenido en consola.

#### `display_function_call(self, name, arguments, output)`
Muestra una llamada a función en formato HTML expandible (`<details>`), con:
- Nombre de la función y argumentos abreviados.
- Argumentos completos.
- Salida de la función.

#### `display_response(self, md_content)`
Convierte contenido en formato Markdown a HTML y lo muestra con estilo en la salida del notebook.




In [5]:
class IPythonChatInterface:

    def input(self):
        question = input('User:').strip()
        return question

    def display(self, content):
        print(content)

    def display_function_call(self, name, arguments, output):
        short_arguments = shorten(arguments)

        call_html = f"""
            <details>
                <summary>Function call: <tt>{name}({short_arguments})</tt></summary>
                <div>
                    <b>Call</b>
                    <pre>{arguments}</pre>
                </div>
                <div>
                    <b>Output</b>
                    <pre>{output}</pre>
                </div>
            </details>
        """
        display(HTML(call_html))

    def display_response(self, md_content):
        html_content = markdown.markdown(md_content)

        html = f"""
            <div>
                <div><b>Assistant:</b></div>
                <div>{html_content}</div>
            </div>
        """
        display(HTML(html))

### Clase `ChatAssistant`

Orquesta la interacción entre el usuario, las herramientas y el modelo de OpenAI.

#### `__init__(self, tools, developer_prompt, interface, openai_client)`
Inicializa:
- `tools`: instancia de `Tools` con las funciones disponibles.
- `developer_prompt`: mensaje inicial para el modelo.
- `interface`: interfaz para entrada/salida (por ejemplo, `IPythonChatInterface`).
- `openai_client`: cliente de OpenAI para generar respuestas.

#### `run(self)`
Bucle principal de interacción:
1. Agrega el `developer_prompt` como primer mensaje.
2. Pide una pregunta al usuario.
3. Si el usuario escribe `"stop"`, finaliza el chat.
4. Envía la conversación al modelo junto con las herramientas disponibles.
5. Procesa la respuesta:
   - Si es texto (`message`), lo muestra como Markdown.
   - Si es una llamada a función (`function_call`):
     - Ejecuta la función.
     - Muestra los argumentos y la salida.
     - Agrega la respuesta al historial.
6. Repite el proceso hasta que no haya más llamadas a funciones.

In [6]:
class ChatAssistant:

    def __init__(self, tools: Tools, developer_prompt: str, interface: IPythonChatInterface, openai_client: OpenAI):
        self.tools = tools
        self.developer_prompt = developer_prompt
        self.interface = interface
        self.openai_client = openai_client

    def run(self) -> None:
        chat_messages = [
            {"role": "developer", "content": self.developer_prompt},
        ]
        
        while True: # Q&A loop
            question = self.interface.input()
        
            if question.lower() == 'stop':
                self.interface.display('chat ended')
                break
        
            chat_messages.append({"role": "user", "content": question})
        
            while True:
                response = self.openai_client.responses.create(
                    model='gpt-4o-mini',
                    input=chat_messages,
                    tools=self.tools.get_tools()
                )
        
                has_function_call = False
                
                for entry in response.output:    
                    chat_messages.append(entry)
                
                    if entry.type == 'message':
                        md_content = entry.content[0].text
                        self.interface.display_response(md_content)
        
                    if entry.type == 'function_call':
                        call_output = self.tools.function_call(entry)
        
                        name = entry.name
                        arguments = entry.arguments
                        output = call_output['output']
                        self.interface.display_function_call(name, arguments, output)
        
                        chat_messages.append(call_output)
                        has_function_call = True
        
                if not has_function_call:
                    break

### Resumen del flujo general

1. **Registro de funciones** con `Tools` para que el modelo pueda llamarlas.
2. **Ejecución del asistente** con `ChatAssistant`, que maneja:
   - Entrada de usuario.
   - Llamadas al modelo de OpenAI.
   - Ejecución de herramientas solicitadas por el modelo.
3. **Interfaz IPython** (`IPythonChatInterface`) para mostrar resultados enriquecidos en Jupyter Notebook.

### Poniendo en practica todo lo anterior

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()
     
if not os.getenv("OPENAI_API_KEY"):
    print("Error: La variable de entorno OPENAI_API_KEY no está definida.")

print("OPENAI_API_KEY cargada correctamente.")

OPENAI_API_KEY cargada correctamente.


In [12]:
import requests

class WeatherService:
    """
    Servicio para obtener información meteorológica actual y por hora
    usando la API de Open-Meteo.
    """

    def get_current_temperature(self, latitude: float, longitude: float) -> float:
        """
        Obtiene la temperatura actual en grados Celsius para una ubicación
        específica usando coordenadas geográficas.

        Args:
            latitude (float): Latitud de la ubicación.
            longitude (float): Longitud de la ubicación.

        Returns:
            float: Temperatura actual en grados Celsius.
        """
        url = (
            "https://api.open-meteo.com/v1/forecast"
            f"?latitude={latitude}&longitude={longitude}"
            "&current=temperature_2m,wind_speed_10m"
            "&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
        )
        response = requests.get(url)
        data = response.json()
        return data['current']['temperature_2m']


In [14]:
from openai import OpenAI

# Cliente OpenAI
client = OpenAI()

# Configuración
tools = Tools()
tiempo = WeatherService()
tools.add_tool(tiempo.get_current_temperature)

iface = IPythonChatInterface()

assistant = ChatAssistant(
    tools=tools,
    developer_prompt="Eres un asistente que puede usar funciones para responder.",
    interface=iface,
    openai_client=client
)

assistant.run()

User: Cual es la temperatura en Asuncion/Paraguay


User: Cuál es la temperatura en Buenos Aires/Argentina


User: stop


chat ended
